<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#ВЫВОД" data-toc-modified-id="ВЫВОД-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>ВЫВОД</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#ВЫВОД" data-toc-modified-id="ВЫВОД-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>ВЫВОД</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Для защиты данных клиентов страховой компании «Хоть потоп» необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

**Основная цель** - защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Описание данных:**


- *Признаки:* пол, возраст и зарплата застрахованного, количество членов его семьи.

- *Целевой признак:* количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
from sklearn.datasets import make_spd_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy.linalg import inv, cond
import numpy as np

In [2]:
try:
    insurance = pd.read_csv('/datasets/insurance.csv')
except:
    insurance = pd.read_csv('/insurance.csv')

In [3]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
insurance.head(20)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [5]:
insurance.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


### ВЫВОД

Предоставленные данные состоят из 5 столбцов и 5000 наблюдений. Все типы определены верно, пропуски и выбросы отсутствуют. Предобработка данных не нужна.

Для дальнейшей работы нам понадобится **Целевой признак** - это столбец с данными о страховых выплатах. Остальные четыре столбца являются признаками. 

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответим на вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

Докажем, что:

$$
a = Xw = X'w' = XPw' = a'
$$

Чтобы это равенство выполнялось, необходимо соблюсти условие:

$$
w' = P^{-1}w
$$

Получим следующее равенство:

$$
a = Xw = X'(P^{-1}w) = XP(P^{-1}w) = XEw = Xw = a
$$

**Ответ:** Нет, не изменится.

**Обоснование:**  Как известно, обратная для квадратной матрицы P — матрица P с верхним индексом -1, произведение которой на P равно единичной матрице. Если любую матрицу умножить на единичную (или наоборот), получится эта же матрица. Поэтому качество линейной регрессии не изменится - матрицы будут одинаковы.

## Алгоритм преобразования

**Алгоритм**

Создадим случайную обратимую матрицу P, затем умножим матрицу с признаками X на матрицу P.

$$
X@P = Z
$$

Для возврата к исходной матрице X, умножим новую матрицу Z на обратную матрицу P^-1.

$$
X@P^{-1} = X
$$

**Обоснование**

Как было доказано в шаге 2, качество линейной регрессии не изменится при умножении матрицы признаков на любую обратимую матрицу P. Поэтому можно следовать описанному выше алгоритму без опасения получить другой результат линейной регрессии.

## Проверка алгоритма

Выделим из датасета набор признаков и целевой признак.

In [6]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

Получим матрицу из значений набора признаков.

In [7]:
features_matrix = features.values

In [8]:
features_matrix 

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Создадим рандомную матрицу нужного размера с помощью инструмента make_spd_matrix библиотеки sklearn.

In [9]:
random_matrix = make_spd_matrix(n_dim=4, random_state=12345)
display(random_matrix)

array([[ 1.37245706, -1.03845957, -0.84389737, -0.26033015],
       [-1.03845957,  2.87886199,  1.67157893,  0.48470484],
       [-0.84389737,  1.67157893,  2.10204907,  0.3257384 ],
       [-0.26033015,  0.48470484,  0.3257384 ,  1.01695329]])

Проверим матрицу на обратимость. При использовании необратимой матрицы дальнейшая работа по описанному алгоритму невозможна. Для проверки будем использовать функцию numpy.linalg.inv(), если матрица необратима, то появится ошибка.

In [10]:
inv(random_matrix)

array([[ 1.06341364,  0.23949912,  0.22304514,  0.08662924],
       [ 0.23949912,  0.72198481, -0.45683306, -0.13647886],
       [ 0.22304514, -0.45683306,  0.93223396, -0.0237669 ],
       [ 0.08662924, -0.13647886, -0.0237669 ,  1.07816747]])

Матрица обратима. Далее перемножим матрицы.

In [11]:
transformed_matrix = features_matrix @ random_matrix
features_transformed = list(transformed_matrix)

Разделим набор признаков и целевой признак исходного датасета на обучающую и тестовую выборки.

In [12]:
features_train, features_test = train_test_split(features, test_size=0.25, random_state=12345)
target_train, target_test = train_test_split(target, test_size=0.25, random_state=12345)

Создадим модели Линейной регрессии:

In [13]:
model1 = LinearRegression()
model2 = LinearRegression()

Обучим первую модель, сделаем предсказание и оценим качество модели.

In [14]:
model1.fit(features_train, target_train)

LinearRegression()

In [15]:
predicted = model1.predict(features_test)
print("R2 для модели 1:", r2_score(target_test, predicted))

R2 для модели 1: 0.435227571270266


Разделим набор признаков преобразованного датасета на обучающую и тестовую выборки. Обучим вторую модель, сделаем предсказание и оценим качество модели.

In [19]:
features_train_transformed, features_test_transformed = train_test_split(
    features_transformed, test_size=0.25, random_state=12345)
model2.fit(features_train_transformed, target_train)
predicted_transformed = model2.predict(features_test_transformed)
print("R2 для модели 2:", r2_score(target_test, predicted_transformed))

R2 для модели 2: 0.4352275712696516


Показатели R2 обеих моделей больше 0, обученная модель неидеальна. Но после преобразования качество модели не ухудшилось, как и требовалось в поставленной задаче.

Попробуем восстановить исходную матрицу, используя формулу из шага 3.

$$
X@P^{-1} = X
$$

In [17]:
final_matrix = transformed_matrix.dot(inv(random_matrix))

In [18]:
#сравним округленные матрицы
np.round(final_matrix) == np.round(features_matrix)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

### ВЫВОД

В ходе исследования было проведено сравнение качества моделей на исходных и преобразованных данных. Мы убедились, что матрицы одинаковы. Таким образом, мы реализовали алгоритм, описанный в шаге 3 и доказали, что качество линейной регрессии не меняется при умножении матрицы признаков на любую обратимую матрицу P.